In [ ]:
from matplotlib import *
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.metrics.pairwise import pairwise_distances


## create fake populations and distances

In [ ]:

# Sample Data: Population of locations (Replace with WorldPop data)
locations = ['A', 'B', 'C', 'D', 'E']
population = [10000, 5000, 20000, 15000, 12000]
distances = {
    ('A', 'B'): 10, ('A', 'C'): 30, ('A', 'D'): 50, ('A', 'E'): 70,
    ('B', 'C'): 20, ('B', 'D'): 40, ('B', 'E'): 60,
    ('C', 'D'): 15, ('C', 'E'): 35,
    ('D', 'E'): 25
}

# Convert to DataFrame
pop_df = pd.DataFrame({'Location': locations, 'Population': population})
dist_df = pd.DataFrame([(k[0], k[1], v) for k, v in distances.items()], columns=['From', 'To', 'Distance'])


## Build your Gravity Model 

The number of trips occurring from i to j are inversely proportional to the distance between i and j, where masses of origin and destination are the respective populations.

<div class="alert alert-info" style="font-size:120%">


The form with the power law deterrence functions is written as:  
$\Large T_{ij} = K \frac{m_i^\alpha m_j^\beta}{d^\gamma}$

The form with the exponential deterrence functions is written as:  
$\Large T_{ij} = K m_i^\alpha m_j^\beta e^{-d/d_0}$

</div>

where $\alpha$ and $\beta$ modulate the attractiveness of masses  
$\gamma$ regulates the decay of the gravity force with distance  
and $d_0$ represents the typical distance travelled by individuals

### create a table containing the two gravity models predicted flows between i and j

In [ ]:

def gravity_model(pop_df, dist_df, alpha=1, beta=1):
    flows = []
    for _, row in dist_df.iterrows():
        pop_i = pop_df.loc[pop_df['Location'] == row['From'], 'Population'].values[0]
        pop_j = pop_df.loc[pop_df['Location'] == row['To'], 'Population'].values[0]
        distance = row['Distance']
        flow = (pop_i ** alpha * pop_j ** beta) / (distance ** 2)
        flows.append(flow)
    dist_df['Gravity Flow'] = flows
    return dist_df

def gravity_model_exp(pop_df, dist_df, alpha=1, beta=1):
    flows = []
    for _, row in dist_df.iterrows():
        pop_i = pop_df.loc[pop_df['Location'] == row['From'], 'Population'].values[0]
        pop_j = pop_df.loc[pop_df['Location'] == row['To'], 'Population'].values[0]
        distance = row['Distance']
        flow = (pop_i ** alpha * pop_j ** beta) * np.exp(-distance/d0)
        flows.append(flow)
    dist_df['Gravity Flow Exp'] = flows
    return dist_df

alpha=1
beta=1
d0=5000
gravity_results = gravity_model(pop_df, dist_df)
gravity_results = gravity_model_exp(pop_df, dist_df)

print("Gravity Model Results:")
print(gravity_results)


## Build your Radiation Model

<div class="alert alert-info" style="font-size:120%">

The number of trips occurring from i to j is controlled by the formula


$\Large T_{ij} = T_i \frac{(m_i m_j)}{(m_i + s_{ij})(m_i + m_j + s_{ij})}$

</div>

where $m_i$ and $m_j$ are the populations of i and j and $s_{ij}$ is the intervening population


### Add the radiation model predicted flows to the table of results

In [ ]:

def radiation_model(pop_df, dist_df):
    flows = []
    for _, row in dist_df.iterrows():
        pop_i = pop_df.loc[pop_df['Location'] == row['From'], 'Population'].values[0]
        pop_j = pop_df.loc[pop_df['Location'] == row['To'], 'Population'].values[0]
        distance = dist_df[(dist_df.From==row['From'])&(dist_df.To==row['To'])].Distance
        loc_s = set(dist_df[dist_df.From==row['From']].groupby(['To']).filter(lambda x:(x['Distance'].max()<distance)&(x['Distance'].max()>0))['To'].values)
        pop_s = pop_df[pop_df.Location.isin(loc_s)]['Population'].sum()
        flow = pop_i * (pop_i * pop_j / ((pop_i + pop_s) * (pop_i + pop_j + pop_s)))
        flows.append(flow)
    dist_df['Radiation Flow'] = flows
    return dist_df

radiation_results = radiation_model(pop_df, dist_df)
print("\nRadiation Model Results:")
print(radiation_results)


## Now model the mobility between Italian provinces  
population from https://demo.istat.it/app/?i=POS 

In [ ]:
pops = pd.read_csv('../data/id_provinces_it.csv').drop(['Unnamed: 0'],axis=1)
#things happen... 'NA' stands for Napoli (Naples), but geopandas reads it as nan...
pops = pops.fillna('NA')
pops.set_index('COD_PROV',inplace=True)
pops.head()

In [ ]:
pop_prov = pd.read_csv('../data/Popolazione residente.csv')
pop_prov = pop_prov[['Codice provincia', 'Totale']]
pop_prov.set_index('Codice provincia', inplace=True)
pop_prov.head()

In [ ]:
populations = pops.merge(pop_prov, how='left', left_on='COD_PROV', right_on='Codice provincia')
#populations.to_csv('population_provinces.csv')
len(populations)

In [ ]:
places = populations.SIGLA

Get mobility data from https://data.humdata.org/dataset/covid-19-mobility-italy   
paper https://www.nature.com/articles/s41597-020-00575-2   
mobility matrix, already normalized across columns => outflows sum to 1 for each patch

In [ ]:
OD = pd.read_csv('../data/od_matrix_daily_flows_norm_full_2020_01_18_2020_06_26.csv')

#from dataframe, create matrix filling empty positions
P = OD.sort_values(['p1','p2']).groupby(['p1','p2']).sum().unstack().fillna(0)
P = P['2020-01-29'].to_numpy()

<div class="alert alert-block alert-warning" style="font-size:120%">

**ATTENTION**  
the mobility matrix encodes transition probabilities was computed on population samples that are not equal to the total populations  

we need to project these probabilities into real provinces population, to do this we will:

* project probabilities to total trips using official population estimates from census
* make the matrix symmetric, assume all trips are round trips

</div>

In [ ]:
# retrieve total flows from origins
p2 = P*np.array(populations.Totale)

# the original matrix is directed, make it symmetric or populations will be mixed!
OD_matrix = np.round((p2+p2.T)/2)

download shapefile of italian provinces from https://public.opendatasoft.com/explore/dataset/georef-italy-provincia/information/

In [ ]:
map_prov = gpd.read_file('../data/georef-italy-provincia/georef-italy-provincia-millesime.shp')
map_prov = map_prov[['prov_sigla','geometry']]
map_prov = map_prov.to_crs(epsg=3003)
map_prov.plot(edgecolor='w',facecolor='lightblue',lw=.2)
plt.axis('off')

In [ ]:
#reorder to match order of data
map_prov = map_prov.set_index("prov_sigla")
map_prov = map_prov.reindex(places)
map_prov = map_prov.reset_index()


In [ ]:
map_prov.head()

remember how we computed the centroid for polygons in geopandas?

In [ ]:
map_prov['centroid'] = map_prov.centroid
ax=map_prov.plot( linewidth=.5, edgecolor='w', color='lightblue')
map_prov['centroid'].plot(color='k',ax=ax, markersize=1)
ax.axis('off')
plt.show()

In [ ]:
italy_prov = map_prov.merge(populations, how='left', left_on='SIGLA', right_on='SIGLA')
italy_prov.head()

In [ ]:
italy_prov['centroid_x'] = italy_prov['centroid'].apply(lambda x: x.x)
italy_prov['centroid_y'] = italy_prov['centroid'].apply(lambda x: x.y)

compute distances in meters (we're in epsg 3003, Monte Mario)

In [ ]:
locations = italy_prov.SIGLA.tolist()
population = italy_prov['Totale'].tolist()

distances = pairwise_distances(italy_prov[['centroid_x','centroid_y']], metric='euclidean')


In [ ]:
np.diag(distances)

In [ ]:
# Convert to DataFrame
pop_df = pd.DataFrame({'Location': locations, 'Population': population})
pop_dict = pop_df.set_index('Location')['Population'].to_dict()
dist_df = pd.DataFrame(distances, index=locations, columns=locations)

In [ ]:
dist_df.head()

In [ ]:
distance_df = dist_df.unstack().reset_index()
distance_df = distance_df.rename(columns={'level_0':'origin','level_1':'destination',0:'distance'})

In [ ]:
distance_df['pop orig'] = distance_df['origin'].apply(lambda x: pop_dict[x])
distance_df['pop dest'] = distance_df['destination'].apply(lambda x: pop_dict[x])

distance_df = distance_df[(distance_df['pop orig']>0) & (distance_df['pop dest']>0)]

### Use your gravity model to generate flows with census data

In [ ]:

# Gravity Model Function
alpha=1
beta=1
gamma=.4
d0=10000 #meters
def Gravity_pow(x, y, d, alpha, beta, gamma):
    if x!=y: #avoid dividing by zero when origin=destination
        return (x**alpha * y**beta) / (d**gamma)
    else: return np.nan

def Gravity_exp(x, y, d, alpha, beta, d0):
    return x**alpha * y**beta * np.exp(-d/d0)

distance_df['gravity model pow'] = distance_df[['pop orig','pop dest','distance']].apply(lambda x: Gravity_pow(x[0],x[1],x[2],alpha,beta,gamma), axis=1)
distance_df['gravity model exp'] = distance_df[['pop orig','pop dest','distance']].apply(lambda x: Gravity_exp(x[0],x[1],x[2],alpha,beta,d0), axis=1)

distance_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
distance_df.head()

In [ ]:
res_df = distance_df.groupby(['origin','destination'])['gravity model pow'].sum().to_frame()
res_df = res_df.reindex(level=0, labels=places).reindex(level=1, labels=places).unstack()
res_matrix = res_df.to_numpy()
res_matrix

In [ ]:
plt.loglog(res_matrix.flatten(),OD_matrix.flatten(),'o', lw=0, markersize=1);
plt.xlabel('predicted trips')
plt.ylabel('observed trips')

In [ ]:
res_matrix_norm = res_matrix/res_matrix.sum(axis=1,keepdims=True)
OD_matrix_norm = OD_matrix/OD_matrix.sum(axis=1,keepdims=True)
plt.loglog(res_matrix_norm.flatten(),OD_matrix_norm.flatten(),'o', lw=0, markersize=1);
plt.loglog([0.00001,1],[0.00001,1], lw=3, ls='--')
plt.ylabel('observed trips')
plt.xlabel('modelled trips')
plt.gca().set_aspect('equal')

**meh!**

We should fit the parameters

The form with the power law deterrence functions is written as:  
$\Large T_{ij} = K \frac{m_i^\alpha m_j^\beta}{d^\gamma}$

If we take the logarithm both left and right of this equation we can linearize the system:  

$\Large log(T_{ij}) = log(K) + \alpha log(m_i) + \beta log(m_j) - \gamma log(d)$

and now this is a simple linear regression! 

Let's take the data and try to model them

In [ ]:
pops = pd.read_csv('../data/id_provinces_it.csv').drop(['Unnamed: 0'],axis=1)
pops = pops.fillna('NA')
pops.set_index('COD_PROV',inplace=True)
pops.head()

In [ ]:
pop_prov = pd.read_csv('../data/Popolazione residente.csv')
pop_prov = pop_prov[['Codice provincia', 'Totale']]
pop_prov.set_index('Codice provincia', inplace=True)
pop_prov.head()

In [ ]:
populations = pops.merge(pop_prov, how='left', left_on='COD_PROV', right_on='Codice provincia')
#populations.to_csv('population_provinces.csv')
populations.head()

data from https://data.humdata.org/dataset/covid-19-mobility-italy   
paper https://www.nature.com/articles/s41597-020-00575-2   
mobility matrix, already normalized across columns => outflows sum to 1 for each patch

In [ ]:
OD = pd.read_csv('../data/od_matrix_daily_flows_norm_full_2020_01_18_2020_06_26.csv')
OD.head()

In [ ]:
new_OD = OD.sort_values(['p1','p2']).groupby(['p1','p2']).sum().reset_index()
new_OD.head()

we have 1,2,3 describing the provinces, but we need tags (TO, VC, etc) to match our table!   
we can use the `pops` table above, reporting numbers and tags  
the numbers are the index of the `pops` table, so we have to use the `.loc` command in the `apply lambda` function

In [ ]:
new_OD['p1'] = new_OD['p1'].apply(lambda x: pops.loc[x].SIGLA)
new_OD['p2'] = new_OD['p2'].apply(lambda x: pops.loc[x].SIGLA)

In [ ]:
new_OD.head()

we can choose one specific week, weeks are represented as columns, so we need to filter out all columns but our desired one  
we can use the `drop` command listing all the columns we don't want or we can use the `[[ ]]` command to restrict the df to a list of columns

In [ ]:
new_OD = new_OD[['p1','p2','2020-01-18']]
new_OD = new_OD.rename(columns={'2020-01-18':'pij'})
new_OD.head()

now we can merge  

In [ ]:
fit_df = distance_df.merge(new_OD, how='left', left_on=['origin','destination'], right_on=['p1','p2'])
fit_df = fit_df.dropna()
fit_df.head()

transform $p_{ij}$ to trips   

In [ ]:
fit_df['trips'] = fit_df['pij']*fit_df['pop orig']

In [ ]:
fit_df.head()

we have 0 observed trips in many origin-destination routes, we are not interested in reproducing them  
we use pandas conditional filtering on the column values  

In [ ]:
filtered_fit_df = fit_df[fit_df.trips!=0]

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# explanatory variables
X = filtered_fit_df[['pop orig', 'pop dest', 'distance']].to_numpy()
# targer variable
y = filtered_fit_df[["trips"]].to_numpy()


apply the link function  

In [ ]:
X = np.log(X)
y = np.log(y)

remember that we had this equation, K is a constant that we want to fit, but it's not in the table  
$log(T_{ij}) = log(K) + \alpha log(m_i) + \beta log(m_j) - \gamma log(d)$

run the linear regression and predict trips using the fitted parameters  
we can use the `fit_intercept=True` command to treat it as the intercept of the model

In [ ]:
regressor = LinearRegression(fit_intercept=True)
regressor.fit(X, y)

y_pred = regressor.predict(X)

we used the log link function, now let's recover the original scale, so...

In [ ]:
y_pred = np.exp(y_pred)
y = np.exp(y)
X = np.exp(X)

we are curious to know what are the parameters of the fit  
in order we had, $\alpha$, $\beta$, $\gamma$

In [ ]:
print('K is',np.exp(regressor.intercept_[0]),' alpha, beta and gamma are', regressor.coef_[0])

let's add the fitted trips to the table, and the predicted $p_{ij}$

In [ ]:
filtered_fit_df['predicted trips'] = y_pred
filtered_fit_df.head()

In [ ]:
plt.loglog(y, y_pred,'o', lw=0, markersize=2)

plt.loglog([0,.2*10**6],[0,.2*10**6], lw=1, ls='--', color='grey')
plt.xlabel('observed trips')
plt.ylabel('modelled trips')
plt.gca().set_aspect('equal')


it looks better, let's see the transition probabilities

In [ ]:
total_predicted_trips = filtered_fit_df.groupby('origin')['predicted trips'].sum().to_frame()
total_predicted_trips.head()

In [ ]:
new_fit_df = filtered_fit_df.merge(total_predicted_trips, how='left', left_on='origin', right_on='origin')
new_fit_df = new_fit_df.rename(columns={'predicted trips_y':'total predicted trips','predicted trips_x':'predicted trips'})
new_fit_df.head()

In [ ]:
new_fit_df['predicted pij'] = new_fit_df['predicted trips']/new_fit_df['total predicted trips']
new_fit_df.head()

In [ ]:
plt.loglog(new_fit_df.pij, new_fit_df['predicted pij'],'o', lw=0, markersize=2)
plt.loglog([0.001,1],[0.001,1], lw=1, ls='--', color='grey')
plt.xlabel('observed trips')
plt.ylabel('modelled trips')
plt.gca().set_aspect('equal')

### Use your radiation model to generate flows with census data

In [ ]:
def get_s(origin,destination):
    df_orig = distance_df[(distance_df.origin==origin)]
    distance = distance_df[(distance_df.origin==origin)&(distance_df.destination==destination)].distance.values[0]
    loc_s = set(df_orig[df_orig['distance']<distance]['destination'].values)
    pop_s = pop_df[pop_df.Location.isin(loc_s)]['Population'].sum()
    return pop_s
        
    
pops_s = []
for i in distance_df.origin.unique():
    print(i)
    for j in distance_df.destination.unique():
        if j>=i:
            pops_s.append([i,j,get_s(i,j)])
            pops_s.append([j,i,get_s(i,j)])

In [ ]:
pop_s = pd.DataFrame(pops_s, columns=['origin','destination','s'])

In [ ]:
new_df = pd.merge(
    left=distance_df, 
    right=pop_s,
    how='left',
    left_on=['origin', 'destination'],
    right_on=['origin', 'destination']
)

In [ ]:

# Radiation Model Function
def radiation_model(pop_i, pop_j, pop_s):
    return pop_i * (pop_i * pop_j / ((pop_i + pop_s) * (pop_i + pop_j + pop_s)))
        

new_df['Radiation model'] = new_df[['pop orig','pop dest','s']].apply(lambda x: radiation_model(x[0],x[1],x[2]),axis=1)


In [ ]:
new_df = new_df.groupby(['origin','destination'])['Radiation model'].sum().to_frame()
new_df = new_df.reindex(level=0, labels=places).reindex(level=1, labels=places).unstack()
new_matrix = new_df.to_numpy()


In [ ]:
new_matrix_norm = new_matrix/new_matrix.sum(axis=1,keepdims=True)
OD_matrix_norm = OD_matrix/OD_matrix.sum(axis=1,keepdims=True)

In [ ]:
plt.loglog(OD_matrix.flatten(),new_matrix.flatten(),'o', lw=0, markersize=1);
plt.loglog([100,1000000],[100,1000000], lw=2, ls='--', color='grey')
plt.xlabel('observed trips')
plt.ylabel('modelled trips')
plt.gca().set_aspect('equal')

In [ ]:
plt.loglog(OD_matrix_norm.flatten(),new_matrix_norm.flatten(),'o', lw=0, markersize=1);
plt.loglog([0.00001,1],[0.00001,1], lw=2, ls='--', color='grey')
plt.xlabel('observed trips')
plt.ylabel('modelled trips')
plt.gca().set_aspect('equal')